# Veri Seti Bağlama

In [1]:
#Drive bağlantısı yapıldı
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
#Kütüphaneler eklendi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential
import tensorflow as tf
from tensorflow import keras


In [3]:
print(os.listdir("/gdrive/MyDrive/Kidney Cancer"))

DATADIR = '/gdrive/MyDrive/Kidney Cancer'

['Normal', 'Tumor']


In [4]:
CATEGORIES = ["Tumor", "Normal"]
IMG_SIZE = 220

In [5]:
all_data = []
def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [6]:
print("Toplam Görüntü Sayısı: ", len(all_data))


Toplam Görüntü Sayısı:  10000


In [7]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

# %80 Train, %20 Test, %10 Validation

In [8]:
from sklearn.model_selection import train_test_split
# 1. Durum: %80 Train, %20 Test, %10 Validation
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)

In [9]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [10]:
#1.Durum
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(7200, 220, 220, 3)
(2000, 220, 220, 3)
(800, 220, 220, 3)
(7200,)
(2000,)
(800,)


#1.CNN MODEL TASARLAMA ,TRAIN,TEST,VALIDATION DEĞERLERİ,TEST VERİSİNİN ACCURACY,PRECISION,RECALL **VE** F1 SCORE,KARMAŞIKLIK MATRİSİ,CLASSIFICATİON REPORT GÖSTERME

In [11]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=2)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=2)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=2)

In [12]:
!pip install keras-tuner
import keras_tuner as kt

In [13]:
from kerastuner.tuners import RandomSearch
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D, Dropout

def build_model(hp):
    model = Sequential()

    model.add(Conv2D(16,(3,3),padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16,(3,3),padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(32, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(64, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3) , padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3,3) , padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Conv2D(128, (3,3), padding="Same",input_shape=(IMG_SIZE,IMG_SIZE,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3,3), padding="Same"))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.27))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dropout(0.27))
    model.add(Dense(1024))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=['accuracy'])
    return model


<ipython-input-13-b59240cffd71>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [14]:
from tensorflow import keras

callback_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='model.h5',
        monitor = 'val_accuracy', save_best_only=True, verbose=3
    ),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=3)
]

In [15]:
#Random Search - Hipermparametre Optimizasyonu
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    directory='my_tuner_dir',
    project_name='kidney_cancer_tuning'
)

Reloading Tuner from my_tuner_dir/kidney_cancer_tuning/tuner0.json


In [16]:
tuner.search(x_train, train_yCl, validation_data=(x_val, valid_yCl),batch_size=32)

In [17]:
#En iyi değeri alma
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

model = tuner.hypermodel.build(best_hps)

In [18]:
#Model Eğitimi
history = model.fit(x_train,
                    train_yCl,
                    validation_data=(x_val, valid_yCl),
                    batch_size=32,
                    epochs=50,
                    callbacks = callback_list,
                    verbose=3)

Epoch 1/50

Epoch 1: val_accuracy improved from -inf to 0.58500, saving model to model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/50

Epoch 2: val_accuracy improved from 0.58500 to 0.90375, saving model to model.h5
Epoch 3/50

Epoch 3: val_accuracy did not improve from 0.90375
Epoch 4/50

Epoch 4: val_accuracy improved from 0.90375 to 0.99625, saving model to model.h5
Epoch 5/50

Epoch 5: val_accuracy did not improve from 0.99625
Epoch 6/50

Epoch 6: val_accuracy did not improve from 0.99625
Epoch 7/50

Epoch 7: val_accuracy did not improve from 0.99625
Epoch 7: early stopping


In [19]:
score_valid = model.evaluate(x_val, valid_yCl)
print("Validation Accuracy: ", score_valid[1])

score_test = model.evaluate(x_test, test_yCl)
print("Validation Accuracy: ", score_test[1])

score_train = model.evaluate(x_train, train_yCl)
print("Validation Accuracy: ", score_train[1])

25/25 [==============================] - 1s 26ms/step - loss: 0.7057 - accuracy: 0.8012
Validation Accuracy:  0.8012499809265137
63/63 [==============================] - 3s 41ms/step - loss: 0.7324 - accuracy: 0.7970
Validation Accuracy:  0.796999990940094
225/225 [==============================] - 7s 30ms/step - loss: 0.6917 - accuracy: 0.8010
Validation Accuracy:  0.8009722232818604


In [20]:
#Test verisinin Accuracy, Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report'u
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Test verisi üzerinde tahmin yapma
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# One-hot encoding'i geri çevirme
y_test_classes = np.argmax(test_yCl, axis=1)

# Accuracy
accuracy = accuracy_score(y_test_classes, y_pred_classes)
print("Test Accuracy: {:.2f}%".format(accuracy * 100))

# Precision
precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
print("Test Precision: {:.2f}".format(precision))

# Recall
recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
print("Test Recall: {:.2f}".format(recall))

# F1 Score
f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')
print("Test F1 Score: {:.2f}".format(f1))

# Karmaşıklık Matrisi
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Confusion Matrix:\n", conf_matrix)

# Classification Report
class_report = classification_report(y_test_classes, y_pred_classes)
print("Classification Report:\n", class_report)



63/63 [==============================] - 3s 30ms/step
Test Accuracy: 79.70%
Test Precision: 0.86
Test Recall: 0.80
Test F1 Score: 0.79
Confusion Matrix:
 [[606 406]
 [  0 988]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.60      0.75      1012
           1       0.71      1.00      0.83       988

    accuracy                           0.80      2000
   macro avg       0.85      0.80      0.79      2000
weighted avg       0.86      0.80      0.79      2000

